In [70]:
library("Seurat")
library("dplyr")
library("hdf5r")
library(SC3)
library(SingleCellExperiment)

In [71]:
hc1= Read10X_h5('/home/cuiyaxuan/spatialLIBD/151676/151676_filtered_feature_bc_matrix.h5')
pbmc=CreateSeuratObject(counts = hc1, project = "HC_1")
pbmc <- FindVariableFeatures(pbmc, selection.method = "vst", nfeatures = 5000)
all.genes <- rownames(pbmc)

Finding variable features for layer counts



In [72]:
mat<-as.matrix(pbmc@assays$RNA$counts)
a <- VariableFeatures(pbmc)
mat=mat[rownames(mat) %in% a,]

In [73]:
sce <- SingleCellExperiment(
      assays = list(
        counts = as.matrix(mat),
        logcounts = log2(as.matrix(mat) + 1)
      )
    )

rowData(sce)$feature_symbol <- rownames(sce)
sce <- sce[!duplicated(rowData(sce)$feature_symbol), ]
sce <- sc3_prepare(sce)
sce <- sc3_calc_dists(sce)
metadata(sce)$sc3$distances[[1]]=metadata(sce)$sc3$distances[[2]]
metadata(sce)$sc3$distances[[3]]=metadata(sce)$sc3$distances[[2]]
sce <- sc3_calc_transfs(sce)


Setting SC3 parameters...

Your dataset contains more than 2000 cells. Adjusting the nstart parameter of kmeans to 50 for faster performance...

Calculating distances between the cells...

Performing transformations and calculating eigenvectors...



In [74]:
k=7
sce <- sc3_kmeans(sce, ks = k)
sce <- sc3_calc_consens(sce)

Performing k-means clustering...



Calculating consensus matrix...



In [75]:
position_path = '/home/cuiyaxuan/spatialLIBD/151676/spatial/tissue_positions_list.csv' 
tissue_local=read.csv(position_path,row.names = 1,header = FALSE)
mat=t(mat)
aa <- rownames(mat)
tissue_local=tissue_local[rownames(tissue_local) %in% aa,]
DF1 <- mutate(tissue_local, id = rownames(tissue_local))
class(tissue_local)
mat=as.data.frame(mat)
DF2 <- mutate(mat, id = rownames(mat))
dat=merge(DF1,DF2,by="id")
dat1=dat[,1:4]
hyperG=matrix(0,dim(mat)[1],dim(mat)[1])
for (i in 1:dim(mat)[1]) {
    x = dat1[i,3]
    y = dat1[i,4]
    for (j in 1:dim(mat)[1]) {
      x1 = dat1[j,3]
      y1 = dat1[j,4]
      radius=(x-x1)^2+(y-y1)^2
      if(radius<=16)
        hyperG[i,j]=1
    }
}

[1] "data.frame"

In [76]:
W=sce@metadata$sc3$consensus[[1]]$consensus

In [77]:
iter<-function(hyperG,W,K){
    hyperG=hyperG*0.9
    hyperG[hyperG==0]=0.1
    spectralClustering <- function(affinity, K, type=3) {
      
      ###This function implements the famous spectral clustering algorithms. There are three variants. The default one is the third type. 
      ###THe inputs are as follows:
      
      #affinity: the similarity matrix;
      #K: the number of clusters
      # type: indicators of variants of spectral clustering 
      
      d = rowSums(affinity)
      d[d == 0] = .Machine$double.eps
      D = diag(d)
      L = D - affinity
      if (type == 1) {
        NL = L
      } else if (type == 2) {
        Di = diag(1 / d)
        NL = Di %*% L
      } else if(type == 3) {
        Di = diag(1 / sqrt(d))
        NL = Di %*% L %*% Di
      }
      eig = eigen(NL)
      res = sort(abs(eig$values),index.return = TRUE)
      U = eig$vectors[,res$ix[1:K]]
      normalize <- function(x) x / sqrt(sum(x^2))
      if (type == 3) {
        U = t(apply(U,1,normalize))
      }
      eigDiscrete = .discretisation(U)
      eigDiscrete = eigDiscrete$discrete
      labels = apply(eigDiscrete,1,which.max)
      
      
      
      return(labels)
    }
    
    
    .discretisation <- function(eigenVectors) {
      
      normalize <- function(x) x / sqrt(sum(x^2))
      eigenVectors = t(apply(eigenVectors,1,normalize))
      
      n = nrow(eigenVectors)
      k = ncol(eigenVectors)
      
      R = matrix(0,k,k)
      R[,1] = t(eigenVectors[round(n/2),])
      
      mini <- function(x) {
        i = which(x == min(x))
        return(i[1])
      }
      
      c = matrix(0,n,1)
      for (j in 2:k) {
        c = c + abs(eigenVectors %*% matrix(R[,j-1],k,1))
        i = mini(c)
        R[,j] = t(eigenVectors[i,])
      }
      
      lastObjectiveValue = 0
      for (i in 1:20) {
        eigenDiscrete = .discretisationEigenVectorData(eigenVectors %*% R)
        
        svde = svd(t(eigenDiscrete) %*% eigenVectors)
        U = svde[['u']]
        V = svde[['v']]
        S = svde[['d']]
        
        NcutValue = 2 * (n-sum(S))
        if(abs(NcutValue - lastObjectiveValue) < .Machine$double.eps) 
          break
        
        lastObjectiveValue = NcutValue
        R = V %*% t(U)
        
      }
      
      return(list(discrete=eigenDiscrete,continuous =eigenVectors))
    }
    
    .discretisationEigenVectorData <- function(eigenVector) {
      
      Y = matrix(0,nrow(eigenVector),ncol(eigenVector))
      maxi <- function(x) {
        i = which(x == max(x))
        return(i[1])
      }
      j = apply(eigenVector,1,maxi)
      Y[cbind(1:nrow(eigenVector),j)] = 1
      
      return(Y)
      
    }
    
    .dominateset <- function(xx,KK=20) {
      ###This function outputs the top KK neighbors.	
      
      zero <- function(x) {
        s = sort(x, index.return=TRUE)
        x[s$ix[1:(length(x)-KK)]] = 0
        return(x)
      }
      normalize <- function(X) X / rowSums(X)
      A = matrix(0,nrow(xx),ncol(xx));
      for(i in 1:nrow(xx)){
        A[i,] = zero(xx[i,]);
        
      }
      
      
      return(normalize(A))
    }
    # Calculate the mutual information between vectors x and y.
    .mutualInformation <- function(x, y) {
      classx <- unique(x)
      classy <- unique(y)
      nx <- length(x)
      ncx <- length(classx)
      ncy <- length(classy)
      
      probxy <- matrix(NA, ncx, ncy)
      for (i in 1:ncx) {
        for (j in 1:ncy) {
          probxy[i, j] <- sum((x == classx[i]) & (y == classy[j])) / nx
        }
      }
      
      probx <- matrix(rowSums(probxy), ncx, ncy)
      proby <- matrix(colSums(probxy), ncx, ncy, byrow=TRUE)
      result <- sum(probxy * log(probxy / (probx * proby), 2), na.rm=TRUE)
      return(result)
    }
    
    # Calculate the entropy of vector x.
    .entropy <- function(x) {
      class <- unique(x)
      nx <- length(x)
      nc <- length(class)
      
      prob <- rep.int(NA, nc)
      for (i in 1:nc) {
        prob[i] <- sum(x == class[i])/nx
      }
      
      result <- -sum(prob * log(prob, 2))
      return(result)
    }
    
    .repmat = function(X,m,n){
      ##R equivalent of repmat (matlab)
      if (is.null(dim(X))) {
        mx = length(X)
        nx = 1
      } else {
        mx = dim(X)[1]
        nx = dim(X)[2]
      }
      matrix(t(matrix(X,mx,nx*n)),mx*m,nx*n,byrow=T)
    }
    
    normalize <- function(X) X / rowSums(X)
    W=normalize(W)
    W = (W+t(W))/2
    hyperG=normalize(hyperG)
    W=as.matrix(hyperG) %*% as.matrix(W) %*% as.matrix(t(hyperG))
    W = (W+t(W))/2
    label<-spectralClustering(W, K)
    return(label)
  }

In [78]:
K=7
label=iter(hyperG,W,K)

In [79]:
true_label=read.csv('/home/cuiyaxuan/spatialLIBD/151676/cluster_labels_151676.csv',row.names=1)

In [80]:
pre_label=label
pre_label[1] 
pre_label=as.data.frame(pre_label)

[1] 5

In [81]:
rownames(pre_label)=rownames(mat)

In [82]:
rownames(pre_label)=rownames(mat)
a <- rownames(pre_label)
aa <- rownames(true_label)
pre_label=pre_label[rownames(pre_label) %in% aa,]

In [83]:
library("mclust")
true_label=as.array(true_label[,1])
ari=adjustedRandIndex(pre_label, true_label)

In [84]:
ari

[1] 0.5247515

In [85]:
#write.csv(label, "label_151672.csv", row.names = TRUE)

In [86]:
n_neigh <- 50
old_type <- label
position <- dat[,3:4]
distance <- as.matrix(dist(position, method = "euclidean"))
n_cell <- nrow(distance)
new_type <- vector("character", n_cell)
for (i in seq_len(n_cell)) {
    vec <- distance[i, ]
    index <- order(vec)  # 按距离排序，获取索引
    neigh_type <- old_type[index[2:(n_neigh + 1)]]  # 获取最近的 n_neigh 个邻居标签
    max_type <- names(sort(table(neigh_type), decreasing = TRUE))[1]  # 找到出现最多的类型
    new_type[i] <- max_type
}
  
# 将结果返回或存入数据框中
new_type <- as.character(new_type)

In [87]:
str(new_type)

 chr [1:3460] "5" "3" "4" "5" "7" "5" "2" "7" "4" "1" "1" "3" "7" "1" "7" ...


In [88]:
true_label=read.csv('/home/cuiyaxuan/spatialLIBD/151676/cluster_labels_151676.csv',row.names=1)
pre_label=new_type
pre_label[1] 
pre_label=as.data.frame(pre_label)
rownames(pre_label)=rownames(mat)
rownames(pre_label)=rownames(mat)
a <- rownames(pre_label)
aa <- rownames(true_label)
pre_label=pre_label[rownames(pre_label) %in% aa,]
pre_label

[1] "5"

[1] "5" "3" "4" "5" "7" "5" "2" "7" "4" "1" "1" "3" "7" "1" "7" "7" "2" "6"
  [19] "4" "7" "2" "3" "5" "4" "4" "7" "1" "3" "3" "4" "3" "1" "1" "1" "5" "2"
  [37] "7" "5" "7" "1" "6" "5" "2" "3" "5" "4" "7" "7" "5" "1" "2" "4" "3" "2"
  [55] "4" "1" "4" "5" "1" "3" "7" "3" "4" "5" "5" "2" "5" "2" "4" "4" "1" "2"
  [73] "4" "3" "3" "4" "4" "2" "1" "3" "1" "1" "3" "2" "1" "2" "7" "5" "7" "3"
  [91] "1" "2" "3" "4" "1" "5" "1" "4" "4" "3" "1" "2" "7" "2" "7" "7" "5" "5"
 [109] "3" "1" "4" "5" "1" "2" "3" "3" "5" "3" "3" "4" "3" "3" "7" "5" "7" "7"
 [127] "7" "2" "4" "1" "7" "7" "2" "4" "7" "2" "4" "7" "7" "1" "1" "5" "1" "2"
 [145] "5" "1" "4" "1" "4" "1" "1" "3" "5" "1" "5" "7" "4" "5" "4" "7" "1" "2"
 [163] "4" "3" "7" "1" "4" "3" "2" "3" "4" "4" "1" "2" "7" "3" "2" "5" "1" "3"
 [181] "5" "3" "3" "7" "2" "3" "4" "2" "1" "4" "2" "4" "7" "1" "3" "1" "5" "1"
 [199] "5" "5" "3" "4" "4" "7" "7" "7" "1" "7" "7" "7" "7" "5" "4" "5" "5" "4"
 [217] "2" "5" "1" "1" "3" "7" "3" "5" "7" "1" "1" "7" "7" "7" "4" "5" "1" "2"
 [235] "1" "1" "5" "4" "2" "1" "3" "4" "1" "5" "7" "3" "1" "2" "3" "4" "2" "4"
 [253] "3" "3" "5" "3" "1" "4" "7" "1" "7" "5" "6" "7" "7" "2" "3" "5" "3" "7"
 [271] "3" "2" "5" "4" "2" "7" "2" "7" "5" "7" "3" "3" "1" "7" "7" "1" "5" "7"
 [289] "5" "4" "7" "5" "6" "4" "4" "1" "2" "5" "2" "3" "7" "7" "2" "2" "1" "7"
 [307] "5" "5" "4" "3" "4" "3" "4" "1" "2" "2" "2" "4" "7" "1" "4" "3" "3" "4"
 [325] "4" "5" "1" "1" "5" "7" "5" "4" "2" "5" "7" "1" "3" "7" "6" "7" "7" "5"
 [343] "2" "3" "7" "2" "1" "7" "6" "7" "3" "4" "4" "3" "7" "5" "1" "2" "6" "7"
 [361] "3" "2" "4" "5" "2" "4" "1" "7" "5" "2" "5" "2" "4" "5" "4" "1" "1" "3"
 [379] "4" "4" "7" "5" "7" "7" "4" "7" "7" "2" "7" "5" "1" "2" "5" "2" "1" "1"
 [397] "3" "7" "7" "3" "3" "2" "1" "3" "1" "3" "7" "1" "1" "1" "1" "7" "3" "1"
 [415] "2" "5" "7" "4" "2" "2" "2" "1" "1" "4" "7" "1" "4" "2" "5" "3" "3" "3"
 [433] "7" "1" "2" "7" "5" "4" "2" "2" "3" "1" "4" "1" "4" "7" "5" "2" "5" "5"
 [451] "3" "5" "5" "3" "7" "7" "3" "7" "7" "5" "1" "1" "2" "5" "4" "7" "4" "7"
 [469] "1" "4" "4" "1" "1" "1" "1" "7" "7" "5" "4" "1" "7" "1" "1" "7" "4" "4"
 [487] "5" "4" "2" "2" "2" "4" "4" "4" "1" "1" "4" "7" "1" "2" "2" "7" "4" "3"
 [505] "7" "2" "3" "3" "3" "7" "2" "4" "1" "5" "5" "2" "1" "7" "5" "5" "7" "1"
 [523] "4" "2" "7" "7" "1" "3" "7" "5" "5" "7" "1" "4" "3" "2" "1" "4" "5" "1"
 [541] "3" "3" "7" "1" "2" "4" "2" "5" "4" "3" "5" "5" "4" "2" "3" "7" "3" "7"
 [559] "2" "1" "4" "1" "4" "1" "7" "3" "5" "4" "1" "5" "7" "3" "5" "7" "1" "2"
 [577] "1" "6" "2" "1" "4" "2" "3" "1" "2" "1" "3" "3" "5" "3" "5" "4" "4" "4"
 [595] "4" "7" "1" "5" "3" "3" "5" "5" "1" "1" "4" "2" "1" "1" "4" "2" "4" "3"
 [613] "1" "5" "6" "4" "1" "5" "2" "1" "6" "1" "5" "4" "1" "1" "1" "1" "5" "3"
 [631] "7" "5" "4" "5" "5" "3" "7" "6" "3" "7" "3" "3" "7" "5" "4" "3" "6" "2"
 [649] "7" "2" "7" "7" "1" "7" "5" "5" "6" "6" "3" "1" "7" "7" "1" "3" "3" "7"
 [667] "3" "7" "4" "1" "7" "1" "7" "2" "1" "1" "1" "7" "1" "1" "5" "3" "7" "5"
 [685] "1" "7" "3" "6" "5" "7" "7" "2" "4" "5" "5" "2" "1" "1" "7" "1" "7" "7"
 [703] "3" "3" "2" "7" "5" "4" "7" "4" "1" "4" "1" "7" "1" "4" "2" "1" "2" "7"
 [721] "2" "5" "7" "7" "2" "6" "7" "4" "6" "7" "6" "6" "1" "3" "1" "5" "2" "7"
 [739] "5" "3" "4" "5" "2" "7" "7" "2" "5" "2" "4" "3" "5" "7" "7" "1" "3" "4"
 [757] "2" "1" "5" "1" "1" "4" "4" "1" "5" "2" "3" "6" "1" "1" "3" "2" "1" "7"
 [775] "1" "2" "7" "5" "3" "7" "2" "2" "7" "1" "1" "4" "2" "2" "4" "1" "4" "4"
 [793] "1" "2" "7" "7" "1" "7" "3" "3" "1" "4" "6" "1" "1" "5" "5" "2" "2" "7"
 [811] "3" "5" "5" "1" "7" "1" "2" "3" "4" "1" "4" "5" "7" "5" "2" "5" "6" "1"
 [829] "5" "2" "5" "1" "5" "3" "2" "2" "7" "2" "3" "3" "5" "4" "1" "5" "7" "5"
 [847] "3" "5" "5" "2" "2" "3" "1" "2" "5" "4" "4" "1" "1" "6" "1" "2" "5" "5"
 [865] "2" "4" "2" "7" "6" "2" "5" "2" "2" "5" "1" "3" "2" "5" "2" "3" "7" "3"
 [883] "7" "1" "1" "1" "7" "6" "5" "3" "1" "1" "2" "3" "4" "3" "5" "3" "1" "2"
 [901] "1" "3" "2" "7" "6" "2" "4" "6" "5" "2" "5" "1

In [89]:
library("mclust")
true_label=as.array(true_label[,1])
ari=adjustedRandIndex(pre_label, true_label)
ari

[1] 0.5185365

In [90]:
true_label <- as.vector(true_label)
pre_label <- as.factor(pre_label)
true_label <- as.factor(true_label)

In [91]:
library(aricode)

# 计算 NMI 和 AMI
nmi <- NMI(pre_label, true_label)
ami <- AMI(pre_label, true_label)

cat("NMI:", nmi, "\n")
cat("AMI:", ami, "\n")

NMI: 0.6415193 
AMI: 0.6405102 


In [92]:
write.csv(new_type, "label_151676.csv", row.names = TRUE)